#### glu層を入れたモデル
image_featの標準化、チャネル正規化, subject_id = 0, im_feat標準化なしin loss  
ConvBlock(in_channels, hid_dim), ConvBlock(hid_dim, hid_dim),  
hid_dim=128, p_drop=0.1, lr=0.001   2975(101)  
hid_dim=128, p_drop=0.1, lr=0.0005  3370(102)  
hid_dim=128, p_drop=0.1, lr=0.0002  3180(103)  
hid_dim=128, p_drop=0.1, lr=0.0001  2391(104)  

hid_dim=64, p_drop=0.1, lr=0.001   3235(105)  
hid_dim=64, p_drop=0.1, lr=0.0005  3307(106)  
hid_dim=64, p_drop=0.1, lr=0.0002  2975(107)  
hid_dim=64, p_drop=0.1, lr=0.0001  2857(108)  

hid_dim=32, p_drop=0.1, lr=0.001   3235(109)  
hid_dim=32, p_drop=0.1, lr=0.0005  3259(110)  
hid_dim=32, p_drop=0.1, lr=0.0002  2999(111)  
hid_dim=32, p_drop=0.1, lr=0.0001  2833(112)  
あまり上がらない...

Dilated conv する


batch_sizeを大きくする